In [20]:
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer
from config import env
import os

In [21]:
diffbot_nlp = DiffbotGraphTransformer(diffbot_api_key=os.getenv("DIFFBOT_API_KEY"))

In [23]:
from langchain_community.document_loaders import WikipediaLoader

query = "Warren Buffett"
raw_documents = WikipediaLoader(query=query).load()
graph_documents = diffbot_nlp.convert_to_graph_documents(raw_documents)

In [24]:
graph_documents[0]

GraphDocument(nodes=[Node(id='http://www.wikidata.org/entity/Q723488', type='Person', properties={'name': 'Howard Homan Buffett', 'positionHeld': 'congressman'}), Node(id='http://www.wikidata.org/entity/Q8033204', type='Organization', properties={'name': 'Jackson-Reed High School'}), Node(id='http://www.wikidata.org/entity/Q451695', type='Person', properties={'name': 'Lesley Stahl'}), Node(id='Alice Deal Junior High School', type='Organization', properties={'name': 'Alice Deal Junior High School'}), Node(id='http://www.wikidata.org/entity/Q47213', type='Person', properties={'name': 'Warren Buffett', 'positionHeld': 'CEO', 'age': '7', 'dateOfBirth': '1930-08-30'}), Node(id='http://www.wikidata.org/entity/Q217583', type='Organization', properties={'name': 'Berkshire Hathaway'}), Node(id='http://www.wikidata.org/entity/Q61', type='Location', properties={'name': 'Washington, D.C.'}), Node(id='Warren', type='Person', properties={'name': 'Warren'}), Node(id='http://www.wikidata.org/entity/Q3

In [25]:
from langchain_neo4j import Neo4jGraph

url = "bolt://localhost:7687"
username = "neo4j"
password = "password"

graph = Neo4jGraph(url=url, username=username, password=password)

In [26]:
graph.add_graph_documents(graph_documents, include_source=True)

In [27]:
graph.refresh_schema()

# Query the graph

In [28]:
from langchain_neo4j import GraphCypherQAChain
from langchain_aws import ChatBedrock

# Create the Cypher and QA LLMs

# Cypher LLM for generating Cypher queries
cypher_llm_bedrock_model = ChatBedrock(
    temperature=0,
    model="us.anthropic.claude-3-5-sonnet-20240620-v1:0",
    region=env.AWS_REGION,
    aws_access_key_id=env.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=env.AWS_SECRET_ACCESS_KEY,
)

# QA LLM for answering questions based on the Cypher query results
qa_llm_bedrock_model = ChatBedrock(
    temperature=0,
    model="us.anthropic.claude-3-5-sonnet-20240620-v1:0",
    region=env.AWS_REGION,
    aws_access_key_id=env.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=env.AWS_SECRET_ACCESS_KEY,
)

chain = GraphCypherQAChain.from_llm(
    cypher_llm=cypher_llm_bedrock_model,
    qa_llm=qa_llm_bedrock_model,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
)

In [30]:
chain.invoke("Who is Charlie Munger?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Charlie Munger"})
RETURN p.name, p.positionHeld, p.dateOfBirth, p.age, p.numberOfChildren, p.academicDegree, p.dateOfDeath
Full Context:
[{'p.name': 'Charlie Munger', 'p.positionHeld': 'investor', 'p.dateOfBirth': None, 'p.age': None, 'p.numberOfChildren': None, 'p.academicDegree': None, 'p.dateOfDeath': None}]

> Finished chain.


{'query': 'Who is Charlie Munger?',
 'result': "Charlie Munger is an investor. Unfortunately, I don't have additional information about his date of birth, age, number of children, academic degree, or date of death."}